In [ ]:
#module 3.0
import urllib
import pandas as pd
import time

def export(input_path,output_path):

    #get pmids
    with open(input_path) as f:
        pmid_ls = f.read().splitlines()
    pmid_ls = sorted(pmid_ls)#sort pmid list
    dic_title ={}#dictionary to store abstracts
    dic_abst = {}#dictionary to store abstracts
    for pmid in pmid_ls:
        dic_title[pmid]="Not found"
        dic_abst[pmid]="Not found"
    i = 0#batch
    while 100*i < len(pmid_ls):#retrieve 100 abstracts in each batch
        html = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids="
        pmid_ls_part = pmid_ls[100*i:min(100*(i+1),len(pmid_ls))]
        for pmid in pmid_ls_part:
            html = html+str(pmid)+","
        html = html[:-1]#remove last ","
        #get contents
        while True:#to prevent HTTP403 error
            try:
                response = urllib.request.urlopen(html)
                break
            except:
                time.sleep(10)
        print("url:", response.geturl())
        content = response.readlines()
        content_str = []#change content datatype to str
        for line in content:
            content_str.append(str(line))
        for text in content_str:
            if "|t|" in text:
                title = text[text.find("|t|")+3:-3]#get after |t| tag and remov \n [:-3]
                dic_title[text[2:text.find("|t|")]]=title
            if "|a|" in text:
                abst = text[text.find("|a|")+3:-3]#get after |a| tag　and remove \n [:-3]
                dic_abst[text[2:text.find("|a|")]]=abst
        i = i+1
        #merge pmid_ls, abs_ls
    df_index =pd.DataFrame(dic_title.keys(),columns=["PMID"],index=dic_title.keys())
    df_title =pd.DataFrame(dic_title.values(),columns=["title"],index=dic_title.keys())
    df_abst =pd.DataFrame(dic_abst.values(),columns=["abstract"],index=dic_abst.keys())
    df=pd.concat([df_index,df_title,df_abst],axis=1)
    df.set_index("PMID", inplace=True)
    display(df)
    df.to_csv(output_path)

    #retrieve PMID not stored in PTC
    df_none = df[(df["title"] == "Not found")|(df["abstract"] == "Not found")]

    print(str(len(pmid_ls)-len(df_none.index))+" titles/abstracts were successfully exported.")
    if df_none.size>0:
        id_none =""
        for id in df_none.index:
            id_none = id_none+id+" "
        print("Publications of "+id_none+" were not found in PTC. Please add titles and abstracts manually.")
    response.close()


input_path = "../pmid.txt"
output_path = "../export.csv"
export(input_path,output_path)